# Models comparisons:

## Loading the models:

In [ ]:
base_model_no_aug = YOLO("runs/detect/base_model/weights/best.pt")

In [ ]:
def plot_metric(run_name, metric_name):
    path = f"runs/detect/{run_name}/results.csv"
    df = pd.read_csv(path)
    plt.plot(df[metric_name], label=run_name)

plt.figure(figsize=(10, 6))
for run in ['baseline', 'aug_flip', 'yolov9e_large']:
    plot_metric(run, 'metrics/mAP_0.5')
plt.legend()
plt.title("Validation mAP@0.5 over epochs")
plt.xlabel("Epoch")
plt.ylabel("mAP@0.5")
plt.grid()
plt.show()

## On vaidation dataset

Looping through all images in validation set, drawing the boxes and saving them

In [ ]:
def draw_predicted_boxes(results, image):
    for result in results:
        boxes = result.boxes.xywh.cpu().numpy()
        confs = result.boxes.conf.cpu().numpy()
        class_ids = result.boxes.cls.cpu().numpy().astype(int)

        for (x_c, y_c, bw, bh), conf, cls in zip(boxes, confs, class_ids):
            x1 = int((x_c - bw / 2))
            y1 = int((y_c - bh / 2))
            x2 = int((x_c + bw / 2))
            y2 = int((y_c + bh / 2))

            color = colors.get(cls, (255, 255, 255))
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

            label = f"{categories.get(cls, 'Unknown')} {conf:.2f}"
            cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    return image

In [ ]:
image_files = [f for f in os.listdir(imgs_val_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

for image_file in image_files:
    image_path = os.path.join(imgs_val_dir, image_file)
    results = base_model_no_aug.predict(source=image_path, conf=0.3)
    image = cv2.imread(image_path) # original image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape
    
    image = draw_predicted_boxes(results, image)
            
    # Saving the img
    cv2.imwrite(f"models_predictions/base_model_no_aug/" + image_file,
                cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    
    # Show image - doesn't work on ubuntu
    # cv2.imshow("Prediction", cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()